In [1]:
import cv2
import numpy as np

In [5]:
class ShapeDetector:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path)
        # self.image = cv2.resize(self.image, (1500, 800))
        self.gray = None
        self.thresh = None
        self.canny = None
        self.contours = None
        self.new_img = None

    def preprocess(self):
        self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        self.gray = cv2.GaussianBlur(self.gray, (5, 5), 0)
        self.thresh = cv2.threshold(self.gray, 200, 255, cv2.THRESH_BINARY)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        self.thresh = cv2.morphologyEx(self.thresh, cv2.MORPH_CLOSE, kernel)
        self.canny = cv2.Canny(self.thresh, 150, 200)

    def detect_shapes(self):
        self.preprocess()
        self.contours, hierarchy = cv2.findContours(self.canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        self.new_img = self.image.copy()
        for cnt in self.contours:
            cv2.drawContours(self.new_img, cnt, -1, (255, 255, 0), 2)
            area = cv2.contourArea(cnt)
            if area > 500:
                peri = cv2.arcLength(cnt, True)
                vertices = cv2.approxPolyDP(cnt, peri * 0.02, True)
                corners = len(vertices)
                x, y, w, h = cv2.boundingRect(vertices)
                cv2.rectangle(self.new_img, (x, y), (x + w, y + h), (0, 255, 255), 2)
                if corners == 3:
                    cv2.putText(self.new_img, "Triangle", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    print("Emergency Triangle Stop was detected, please slow down!")
                elif corners == 4:
                    cv2.putText(self.new_img, "Rectangle", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                elif corners == 5:
                    cv2.putText(self.new_img, "Pentagon", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                elif corners >= 6:
                    cv2.putText(self.new_img, "Circle", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            print("Shapes is undeteced")

    def show_images(self):
        cv2.imshow("Image", self.image)
        cv2.imshow("Threshold", self.thresh)
        cv2.imshow("Canny Image", self.canny)
        cv2.imshow("Contour Image", self.new_img)
        cv2.waitKey(0)


if __name__ == "__main__":
    detector = ShapeDetector("shapes_image/shapes.jpg")
    detector.detect_shapes()
    detector.show_images()


Emergency Triangle Stop was detected, please slow down!
Shapes is undeteced
